In [1]:
import pandas as pd
import wandb
from wd.utils.utils import mlflow_linearize

In [2]:
from IPython.core.display_functions import display


def get_runs(run_filters):
    # Project is specified by <entity/project-name>
    runs = api.runs("pasqualedem/weeddetection", filters=run_filters)

    summary_list, config_list, name_list = [], [], []
    for run in runs:
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        config_list.append(
            {k: v for k,v in run.config.items()
              if not k.startswith('_')})

        # .name is the human-readable name of the run.
        name_list.append(run.name)
    df = pd.DataFrame([{**mlflow_linearize(dc), **mlflow_linearize(ds)} for dc, ds in zip(config_list, summary_list)])
    return df

def remove_hyp(df):
    to_delete = filter(lambda x: x.startswith('hyper_params'), df.columns)
    display(list(to_delete))
    for l in to_delete:
        df.drop(l, inplace=True)

def rename_values(df: pd.DataFrame, vrenamings: dict):
    for column, values in vrenamings.items():
        if column in df.columns:
            df[column] = df[column].replace(values)

In [3]:
api = wandb.Api()

In [201]:
renamings = {
    'in_params.dataset.channels': 'Channels',
    'in_params.train_params.optimizer_params.weight_decay': "Weight Decay",
    'in_params.train_params.initial_lr': 'LR',
    'in_params.model.name': 'Model',
    'in_params.model.params.backbone': 'Backbone',
    'in_params.train_params.freeze_pretrained': 'Strategy',
    'in_params.model.params.main_pretrained': 'Used Weights',
    'train_WrapF1': 'Train F1',
    'f1': 'F1',
    'in_params.dataset.batch_size': 'Batch Size',
    'precision': 'Precision',
    'in_params.train_params.loss.params.gamma': 'Gamma',
    'in_params.model.params.fusion_type': 'DropPath',
    'recall': 'Recall'
}
value_renamings = {
    'Strategy': {True: 'Freeze', False: 'Fine-Tuning'},
    'DropPath': {'conv_sum_drop': 'Yes', 'conv_sum': 'No'}
}
channels_ordering = {
    "G, NDVI, NIR, R, RE" : 1,
    "R, G, NIR, RE": 2,
    "R, G, RE, NIR": 2,
    "R, G, RE": 3,
    "G, NIR, R": 3,
    "R, G, NIR": 4,
    "R, G, NDVI": 5,
    "R, G": 6,
    "CIR": 7,
    "NDVI": 8,

    'Fine-Tuning': 1,
    'Freeze': 2,

    'R': 1,
    'G': 2,
    'RGB': 2.5,
    'G, G, R': 3,
    'R, G, G, G': 4,

    'MiT-B0': 0,
    'MiT-B1': 1,
}

## Lawin

In [ ]:
filters = {
    'group': 'Lawin'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.optimizer_params.weight_decay',
    'in_params.train_params.initial_lr',
    'in_params.model.params.backbone',
    # 'in_params.model.name',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
runs_df

In [ ]:
projected_runs = runs_df[
    selected_cols
]

In [ ]:
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [ ]:
idx = selected_runs.groupby(by='Channels')['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by="Channels", key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)

In [ ]:
selected_runs

In [ ]:
best_runs

In [ ]:
best_runs.to_csv("../../exps/lawin.csv")

In [ ]:
selected_runs.to_csv("../../exps/lawin-complete.csv")

## Lawin Aug

In [ ]:
filters = {
    'group': 'LawinAugPatience'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.optimizer_params.weight_decay',
    'in_params.train_params.initial_lr',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
runs_df

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [ ]:
idx = selected_runs.groupby(by='Channels')['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by="Channels", key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)

In [ ]:
best_runs.to_csv("../../exps/lawin_aug.csv")

In [ ]:
selected_runs.to_csv("../../exps/lawin_aug-complete.csv")

## Lawin Pretrained

In [ ]:
filters = {
    '$or': [{'group': 'LawinPretrained'}, {'group': 'LawinFrozen'}]
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.optimizer_params.weight_decay',
    'in_params.train_params.initial_lr',
    'in_params.train_params.freeze_pretrained',
    'in_params.model.params.backbone',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
runs_df

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [ ]:
idx = selected_runs.groupby(by=['Channels', 'Strategy'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Strategy', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
best_runs.to_csv("../../exps/lawin_pretrained.csv")
selected_runs.to_csv("../../exps/lawin_pretrained-complete.csv")

## Focal

In [ ]:
filters = {
    'group': 'Focal'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.freeze_pretrained',
    'in_params.train_params.loss.params.gamma',
    'in_params.model.params.backbone',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [ ]:
idx = selected_runs.groupby(by=['Channels', 'Strategy'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Strategy', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
best_runs.to_csv("../../exps/focal.csv")
selected_runs.to_csv("../../exps/focal-complete.csv")

## SRR

In [ ]:
filters = {
    'group': 'RandomRotation'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.freeze_pretrained',
    'in_params.train_params.loss.params.gamma',
    'in_params.model.params.backbone',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs
selected_runs.fillna("", inplace=True)

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
idx = selected_runs.groupby(by=['Channels', 'Strategy'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Strategy', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
best_runs.to_csv("../../exps/rotation.csv")
selected_runs.to_csv("../../exps/rotation-complete.csv")

## Double Encoder

In [ ]:
filters = {
    'group': 'DoubleLawin'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.freeze_pretrained',
    'in_params.train_params.loss.params.gamma',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
runs_df

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs
selected_runs.fillna("", inplace=True)

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
idx = selected_runs.groupby(by=['Channels', 'Strategy'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Strategy', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
best_runs.to_csv("../../exps/doublelawin.csv")
selected_runs.to_csv("../../exps/doublelawin-complete.csv")

## Double Encoder Pre-Trained

In [ ]:
filters = {
    'group': 'DoubleLawinPretrained'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.loss.params.gamma',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs
selected_runs.fillna("", inplace=True)

In [ ]:
selected_runs.fillna("", inplace=True)
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [ ]:
selected_runs

In [ ]:
selected_runs.to_csv("../../exps/doublelawinpretrained.csv")

## Double Encoder Pre-Trained Drop

In [ ]:
filters = {
    'group': 'DLPretrainedDrop'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.loss.params.gamma',
    'train_WrapF1',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall', 'Train F1']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.fillna("", inplace=True)
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [ ]:
selected_runs

In [ ]:
selected_runs.to_csv("../../exps/dlpretraineddrop.csv")

## RGB Weights

In [261]:
filters = {
    'group': 'RedEdge'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.model.params.backbone',
    'in_params.train_params.freeze_pretrained',
    'in_params.train_params.loss.params.gamma',
    'in_params.model.params.main_pretrained',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [262]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

['hyper_params.arch_params.schema',
 'hyper_params.arch_params.sync_bn',
 'hyper_params.arch_params.backbone',
 'hyper_params.arch_params.in_channels',
 'hyper_params.arch_params.num_classes',
 'hyper_params.arch_params.out_channels',
 'hyper_params.arch_params.main_channels',
 'hyper_params.arch_params.input_channels',
 'hyper_params.arch_params.main_pretrained',
 'hyper_params.arch_params.output_channels',
 'hyper_params.arch_params.side_pretrained',
 'hyper_params.arch_params.backbone_pretrained',
 'hyper_params.dataset_params.root',
 'hyper_params.dataset_params.size',
 'hyper_params.dataset_params.schema',
 'hyper_params.dataset_params.s3_link',
 'hyper_params.dataset_params.channels',
 'hyper_params.dataset_params.hor_flip',
 'hyper_params.dataset_params.ver_flip',
 'hyper_params.dataset_params.crop_size',
 'hyper_params.dataset_params.batch_size',
 'hyper_params.dataset_params.dataset_dir',
 'hyper_params.dataset_params.num_classes',
 'hyper_params.dataset_params.num_workers',
 

In [263]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [266]:
selected_runs.fillna(0, inplace=True)
selected_runs.rename(columns=renamings, inplace=True)
selected_runs['Gamma'] = selected_runs['Gamma'].astype(float).astype(int)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs.sort_values(by=['Backbone', 'Channels', 'Strategy', 'Used Weights'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)

In [267]:
selected_runs

,Channels,Backbone,Strategy,Gamma,Used Weights,F1,Precision,Recall
20,"R, G, NIR, RE",MiT-B0,Fine-Tuning,2,"R, G, G, G",0.447,0.711,0.433
22,"R, G, NIR, RE",MiT-B0,Fine-Tuning,0,"R, G, G, G",0.616,0.653,0.697
21,"R, G, NIR, RE",MiT-B0,Freeze,2,"R, G, G, G",0.499,0.612,0.452
23,"R, G, NIR, RE",MiT-B0,Freeze,0,"R, G, G, G",0.534,0.496,0.753
16,CIR,MiT-B0,Fine-Tuning,2,"G, G, R",0.525,0.640,0.510
17,CIR,MiT-B0,Fine-Tuning,0,"G, G, R",0.603,0.589,0.756
12,CIR,MiT-B0,Freeze,2,"G, G, R",0.527,0.629,0.481
13,CIR,MiT-B0,Freeze,0,"G, G, R",0.545,0.517,0.767
18,NDVI,MiT-B0,Fine-Tuning,2,G,0.572,0.607,0.634
19,NDVI,MiT-B0,Fine-Tuning,0,G,0.577,0.503,0.923


In [268]:
selected_runs.to_csv("../../exps/rgb_weights-complete.csv")

In [275]:
filters = {
    '$or': [{'group': 'RedEdge'}, {'group': 'RandomRotation'}]
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.model.params.backbone',
    'in_params.train_params.freeze_pretrained',
    'in_params.model.params.main_pretrained',
    'f1',
]
roundings = ['F1']

In [292]:
runs_df = get_runs(filters)
remove_hyp(runs_df)
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

['hyper_params.arch_params.schema',
 'hyper_params.arch_params.sync_bn',
 'hyper_params.arch_params.backbone',
 'hyper_params.arch_params.in_channels',
 'hyper_params.arch_params.num_classes',
 'hyper_params.arch_params.out_channels',
 'hyper_params.arch_params.main_channels',
 'hyper_params.arch_params.input_channels',
 'hyper_params.arch_params.main_pretrained',
 'hyper_params.arch_params.output_channels',
 'hyper_params.arch_params.side_pretrained',
 'hyper_params.arch_params.backbone_pretrained',
 'hyper_params.dataset_params.root',
 'hyper_params.dataset_params.size',
 'hyper_params.dataset_params.schema',
 'hyper_params.dataset_params.s3_link',
 'hyper_params.dataset_params.channels',
 'hyper_params.dataset_params.hor_flip',
 'hyper_params.dataset_params.ver_flip',
 'hyper_params.dataset_params.crop_size',
 'hyper_params.dataset_params.batch_size',
 'hyper_params.dataset_params.dataset_dir',
 'hyper_params.dataset_params.num_classes',
 'hyper_params.dataset_params.num_workers',
 

In [293]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs.loc[(selected_runs['Channels'] == 'CIR') & selected_runs['Used Weights'].isna(), 'Used Weights'] = 'RGB'
selected_runs.loc[(selected_runs['Channels'].isin([['NDVI']])) & selected_runs['Used Weights'].isna(), 'Used Weights'] = 'R'
selected_runs.drop(selected_runs[selected_runs['Channels'].isin([['R', 'G']])].index, inplace=True)

C:\Users\pasqu\AppData\Local\Temp\ipykernel_11100\1874336447.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_11100\1874336447.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.drop(selected_runs[selected_runs['Channels'].isin([['R', 'G']])].index, inplace=True)


In [294]:
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs.sort_values(by=['Backbone', 'Channels', 'Used Weights'],
                          key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)

C:\Users\pasqu\AppData\Local\Temp\ipykernel_11100\2999623128.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].replace(values)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_11100\3723124597.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs[roundings] = selected_runs[roundings].round(5)


In [304]:
idx = selected_runs.groupby(by=['Channels', 'Strategy', 'Used Weights'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Channels', 'Strategy'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)

C:\Users\pasqu\AppData\Local\Temp\ipykernel_11100\3340784096.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_runs.sort_values(by=['Channels', 'Strategy'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_11100\3340784096.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_runs[roundings] = best_runs[roundings].round(3)


In [305]:
best_runs

,Channels,Backbone,Strategy,Used Weights,F1
22,"R, G, NIR, RE",MiT-B0,Fine-Tuning,"R, G, G, G",0.616
23,"R, G, NIR, RE",MiT-B0,Freeze,"R, G, G, G",0.534
30,CIR,MiT-B1,Fine-Tuning,RGB,0.650
6,CIR,MiT-B1,Fine-Tuning,"G, G, R",0.669
13,CIR,MiT-B0,Freeze,"G, G, R",0.545
27,CIR,MiT-B1,Freeze,RGB,0.602
37,NDVI,MiT-B0,Fine-Tuning,R,0.594
8,NDVI,MiT-B1,Fine-Tuning,G,0.645
28,NDVI,MiT-B1,Freeze,R,0.685
9,NDVI,MiT-B1,Freeze,G,0.656


In [321]:
compare_runs = best_runs.merge(best_runs, on=['Channels', 'Strategy'])
compare_runs.drop(compare_runs[compare_runs['Used Weights_x'] == compare_runs['Used Weights_y']].index, inplace=True)
compare_runs.drop(compare_runs[compare_runs['Used Weights_x'] == 'G, G, R'].index, inplace=True)
compare_runs.drop(compare_runs[compare_runs['Used Weights_x'] == 'G'].index, inplace=True)
compare_runs

,Channels,Backbone_x,Strategy,Used Weights_x,F1_x,Backbone_y,Used Weights_y,F1_y
3,CIR,MiT-B1,Fine-Tuning,RGB,0.650,MiT-B1,"G, G, R",0.669
8,CIR,MiT-B1,Freeze,RGB,0.602,MiT-B0,"G, G, R",0.545
11,NDVI,MiT-B0,Fine-Tuning,R,0.594,MiT-B1,G,0.645
15,NDVI,MiT-B1,Freeze,R,0.685,MiT-B1,G,0.656


In [322]:
complete = best_runs[best_runs['Channels'] == 'R, G, NIR, RE']
complete.rename(columns={'Gamma': 'Gamma_y', 'F1': 'F1_y', 'Used Weights': 'Used Weights_y', 'Backbone': 'Backbone_x'}, inplace=True)
compare_runs = compare_runs.append(complete)
compare_runs.fillna("", inplace=True)
compare_runs

C:\Users\pasqu\AppData\Local\Temp\ipykernel_11100\4110542001.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complete.rename(columns={'Gamma': 'Gamma_y', 'F1': 'F1_y', 'Used Weights': 'Used Weights_y', 'Backbone': 'Backbone_x'}, inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_11100\4110542001.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  compare_runs = compare_runs.append(complete)


,Channels,Backbone_x,Strategy,Used Weights_x,F1_x,Backbone_y,Used Weights_y,F1_y
3,CIR,MiT-B1,Fine-Tuning,RGB,0.65,MiT-B1,"G, G, R",0.669
8,CIR,MiT-B1,Freeze,RGB,0.602,MiT-B0,"G, G, R",0.545
11,NDVI,MiT-B0,Fine-Tuning,R,0.594,MiT-B1,G,0.645
15,NDVI,MiT-B1,Freeze,R,0.685,MiT-B1,G,0.656
22,"R, G, NIR, RE",MiT-B0,Fine-Tuning,,,,"R, G, G, G",0.616
23,"R, G, NIR, RE",MiT-B0,Freeze,,,,"R, G, G, G",0.534


In [324]:
compare_runs.to_csv("../../exps/rgb_weights.csv")

## Laweed

In [331]:
filters = {
    'group': 'Laweed'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.loss.params.gamma',
    'in_params.train_params.freeze_pretrained',
    'in_params.model.params.main_pretrained',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [332]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

['hyper_params.arch_params.schema',
 'hyper_params.arch_params.sync_bn',
 'hyper_params.arch_params.backbone',
 'hyper_params.arch_params.in_channels',
 'hyper_params.arch_params.num_classes',
 'hyper_params.arch_params.out_channels',
 'hyper_params.arch_params.main_channels',
 'hyper_params.arch_params.input_channels',
 'hyper_params.arch_params.main_pretrained',
 'hyper_params.arch_params.output_channels',
 'hyper_params.arch_params.side_pretrained',
 'hyper_params.arch_params.backbone_pretrained',
 'hyper_params.dataset_params.root',
 'hyper_params.dataset_params.size',
 'hyper_params.dataset_params.schema',
 'hyper_params.dataset_params.s3_link',
 'hyper_params.dataset_params.channels',
 'hyper_params.dataset_params.hor_flip',
 'hyper_params.dataset_params.ver_flip',
 'hyper_params.dataset_params.crop_size',
 'hyper_params.dataset_params.batch_size',
 'hyper_params.dataset_params.dataset_dir',
 'hyper_params.dataset_params.num_classes',
 'hyper_params.dataset_params.num_workers',
 

In [333]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [334]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs.fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
idx = selected_runs.groupby(by=['Channels', 'Strategy'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Strategy', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

C:\Users\pasqu\AppData\Local\Temp\ipykernel_11100\1084863829.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_11100\2999623128.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].replace(values)
C:\Users\pasqu\AppData\Local\Temp\ipykernel_11100\1084863829.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [335]:
selected_runs

,Channels,Gamma,Strategy,Used Weights,F1,Precision,Recall
0,"R, G, NIR, RE",2.0,Fine-Tuning,"R, G, G, G",0.444,0.656,0.429
1,"R, G, NIR, RE",2.0,Freeze,"R, G, G, G",0.489,0.616,0.449
2,"R, G, NIR, RE",0.0,Fine-Tuning,"R, G, G, G",0.603,0.593,0.774
3,"R, G, NIR, RE",0.0,Freeze,"R, G, G, G",0.530,0.494,0.773
4,CIR,2.0,Fine-Tuning,0,0.512,0.705,0.475
5,CIR,2.0,Freeze,0,0.561,0.686,0.502
6,CIR,0.0,Fine-Tuning,0,0.622,0.622,0.702
7,CIR,0.0,Freeze,0,0.592,0.532,0.817
8,NDVI,2.0,Fine-Tuning,G,0.428,0.663,0.472
9,NDVI,2.0,Freeze,G,0.567,0.703,0.508


In [336]:
best_runs

,Channels,Gamma,Strategy,Used Weights,F1,Precision,Recall
2,"R, G, NIR, RE",0.0,Fine-Tuning,"R, G, G, G",0.603,0.593,0.774
6,CIR,0.0,Fine-Tuning,0,0.622,0.622,0.702
10,NDVI,0.0,Fine-Tuning,G,0.587,0.523,0.870
3,"R, G, NIR, RE",0.0,Freeze,"R, G, G, G",0.530,0.494,0.773
7,CIR,0.0,Freeze,0,0.592,0.532,0.817
9,NDVI,2.0,Freeze,G,0.567,0.703,0.508


In [ ]:
selected_runs.to_csv("../../exps/laweed-complete.csv")

## Double Laweed

In [ ]:
filters = {
    '$or': [{'group': 'DoubleLaweedPretrained'}, {'group': 'DoubleLaweedPrBatch'}, {'group': 'DoubleLaweedPrDropBatch'}]
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.loss.params.gamma',
    'in_params.model.params.fusion_type',
    'in_params.dataset.batch_size',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs['DropPath'].fillna('conv_sum', inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs['Gamma'].fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

idx = selected_runs.groupby(by=['Channels'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
selected_runs